# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

In [1]:
#importing some useful packages
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [2]:
def plotLaneLine(image):
    print('This image is:', type(image), 'with dimesions:', image.shape)

    ysize = image.shape[0]
    xsize = image.shape[1]
    color_select = np.copy(image)
    line_image = np.copy(image)

    red_threshold = 120
    green_threshold = 120
    blue_threshold = 120

    rgb_threshold = [red_threshold, green_threshold, blue_threshold]


    left_bottom = [200, 539]
    right_bottom = [930, 539]
    apex = [485, 300]

    fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
    fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

    # Mask pixels below the threshold
    color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                        (image[:,:,1] < rgb_threshold[1]) | \
                        (image[:,:,2] < rgb_threshold[2])

    # Find the region inside the lines
    XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
    region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                        (YY > (XX*fit_right[0] + fit_right[1])) & \
                        (YY < (XX*fit_bottom[0] + fit_bottom[1]))

    color_select[color_thresholds] = [0, 0, 0]
    line_image[~color_thresholds & region_thresholds] = [255, 0, 0]

    # Display the image and show region and color selections
    x = [left_bottom[0], right_bottom[0], apex[0], left_bottom[0]]
    y = [left_bottom[1], right_bottom[1], apex[1], left_bottom[1]]
    gray = cv2.cvtColor(color_select,cv2.COLOR_RGB2GRAY)

    # Define a kernel size for Gaussian smoothing / blurring
    kernel_size = 1 # Must be an odd number (3, 5, 7...)
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)
    # Define our parameters for Canny and run it
    low_threshold = 50
    high_threshold = 150


    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    #vertices = np.array([[(0,imshape[0]),(0, 0), (imshape[1], 0), (imshape[1],imshape[0])]], dtype=np.int32)
    vertices = np.array([[(0,imshape[0]),(450, 310), (490, 310), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = 1*np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 30    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 5 #minimum number of pixels making up a line
    max_line_gap = 200   # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # Iterate over the output "lines" and draw lines on a blank image
    draw_lines(line_image,lines, [255,0,0],4)

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(image, 0.8, line_image, 1, 0) 


    return lines_edges


**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [3]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    x1Left=[]
    y1Left=[]
    x2Left=[]
    y2Left=[]
    x1Right=[]
    y1Right=[]
    x2Right=[]
    y2Right=[]
    aLeftx1=aLefty1=aLeftx2=aLefty2=aRightx1=aRighty1=aRightx2=aRighty2=0
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope=(y2-y1)/(x2-x1)
            if(slope<-0.5):
                x1Left.append(x1)
                y1Left.append(y1)
                x2Left.append(x2)
                y2Left.append(y2)
                print("slope:", slope)
                print("x1:", x1, "y1:",y1)
                print("x2:", x2, "y2:",y2)
            if(slope>0.5):
                x1Right.append(x1)
                y1Right.append(y1)
                x2Right.append(x2)
                y2Right.append(y2)
                print("slope:", slope)
                print("x1:", x1, "y1:",y1)
                print("x2:", x2, "y2:",y2)
    aLeftx1=average(x1Left)
    aLefty1=average(y1Left)
    aLeftx2=average(x2Left)
    aLefty2=average(y2Left)
    aRightx1=average(x1Right)
    aRighty1=average(y1Right)
    aRightx2=average(x2Right)
    aRighty2=average(y2Right)
    
    if((aLeftx2-aLeftx1)!=0):
        slopeLeft=(aLefty2-aLefty1)/(aLeftx2-aLeftx1)
    else:
        slopeLeft=1
    if((aRightx2-aRightx1)!=0):  
        slopeRight=(aRighty2-aRighty1)/(aRightx2-aRightx1)
    else:
        slopeRight=1
    
    
    
    bLeft=aLefty1-(slopeLeft*aLeftx1)
    bRight=aRighty1-(slopeRight*aRightx1)
    
    
    
    
    #if y=540
    xBotLeftLane=(540-bLeft)/slopeLeft
    yBotRightLane=(540-bRight)/slopeRight
    
    xIntersectionLanes=(bRight-bLeft)/(slopeLeft-slopeRight)
    yIntersectionLanes=(slopeLeft*xIntersectionLanes)+bLeft
    
    #cv2.line(img, (int(xBotLeftLane), 540), (aLeftx2, aLefty2), color, thickness)
    #cv2.line(img, (aRightx1, aRighty1), (int(yBotRightLane), 540), color, thickness)      
                
    cv2.line(img, (int(xBotLeftLane), 540), (int(xIntersectionLanes), int(yIntersectionLanes)), color, thickness)
    cv2.line(img, (int(xIntersectionLanes), int(yIntersectionLanes)), (int(yBotRightLane), 540), color, thickness)      
    
def average(List):
    Length=len(List)
    Sum = 0
    for l in List:
        Sum=Sum+l
    if(Sum>0):
        return int(Sum/Length)
    else: 
        return 0
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
import os
from pylab import figure, axes, pie, title, show
from PIL import Image
os.listdir("test_images/")

['resultsolidWhiteCurve.jpg',
 'resultsolidWhiteRight.jpg',
 'resultsolidYellowCurve.jpg',
 'resultsolidYellowCurve2.jpg',
 'resultsolidYellowLeft.jpg',
 'resultwhiteCarLaneSwitch.jpg',
 'solidWhiteCurve.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg']

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
for x in os.listdir("test_images/"):
    print(x)
    imageName="test_images/"+x
    resultImageName="test_images/result"+x
    image = mpimg.imread(imageName)
    image = plotLaneLine(image)
    plt.imshow(image)
    im = Image.fromarray(image)
    im.save(resultImageName)
    #plt.save(resultImageName)
    #scipy.misc.imsave(resultImageName, plt
    #fig = image.figure()
    #image.imsave('test.jpg')


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2


In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    return result

Let's try the one with the solid white lane on the right first ...

In [7]:
white_output = 'challenge1.mp4'
clip1 = VideoFileClip("challenge.mp4")
white_clip = clip1.fl_image(plotLaneLine) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

This image is: <class 'numpy.ndarray'> with dimesions: (720, 1280, 3)
slope: 0.57957957958
x1: 758 y1: 482
x2: 1091 y2: 675
slope: 0.649746192893
x1: 674 y1: 422
x2: 1068 y2: 678
slope: 0.555555555556
x1: 815 y1: 519
x2: 1058 y2: 654
slope: -0.555555555556
x1: 554 y1: 486
x2: 671 y2: 421
slope: -0.577181208054
x1: 490 y1: 526
x2: 639 y2: 440
slope: 0.615384615385
x1: 1021 y1: 645
x2: 1073 y2: 677
slope: -0.649572649573
x1: 246 y1: 688
x2: 597 y2: 460
slope: 1.07194244604
x1: 696 y1: 421
x2: 974 y2: 719
slope: 0.64
x1: 1035 y1: 654
x2: 1060 y2: 670
slope: 0.752808988764
x1: 938 y1: 583
x2: 1027 y2: 650
slope: 0.7
x1: 687 y1: 421
x2: 1057 y2: 680
slope: 0.649746192893
x1: 672 y1: 422
x2: 1066 y2: 678
slope: 0.53164556962
x1: 918 y1: 577
x2: 997 y2: 619
slope: -0.525
x1: 590 y1: 464
x2: 630 y2: 443
slope: 0.647798742138
x1: 717 y1: 449
x2: 1035 y2: 655
slope: 0.556962025316
x1: 816 y1: 517
x2: 1053 y2: 649
[MoviePy] >>>> Building video challenge1.mp4
[MoviePy] Writing video challenge1.mp4

  0%|                                                                                                                | 0/251 [00:00<?, ?it/s]

This image is: <class 'numpy.ndarray'> with dimesions: (720, 1280, 3)
slope: 0.57957957958
x1: 758 y1: 482
x2: 1091 y2: 675
slope: 0.649746192893
x1: 674 y1: 422
x2: 1068 y2: 678
slope: 0.555555555556
x1: 815 y1: 519
x2: 1058 y2: 654
slope: -0.555555555556
x1: 554 y1: 486
x2: 671 y2: 421
slope: -0.577181208054
x1: 490 y1: 526
x2: 639 y2: 440
slope: 0.615384615385
x1: 1021 y1: 645
x2: 1073 y2: 677
slope: -0.649572649573
x1: 246 y1: 688
x2: 597 y2: 460
slope: 1.07194244604
x1: 696 y1: 421
x2: 974 y2: 719
slope: 0.64
x1: 1035 y1: 654
x2: 1060 y2: 670
slope: 0.752808988764
x1: 938 y1: 583
x2: 1027 y2: 650
slope: 0.7
x1: 687 y1: 421
x2: 1057 y2: 680
slope: 0.649746192893
x1: 672 y1: 422
x2: 1066 y2: 678
slope: 0.53164556962
x1: 918 y1: 577
x2: 997 y2: 619
slope: -0.525
x1: 590 y1: 464
x2: 630 y2: 443
slope: 0.647798742138
x1: 717 y1: 449
x2: 1035 y2: 655
slope: 0.556962025316
x1: 816 y1: 517
x2: 1053 y2: 649


  0%|▍                                                                                                       | 1/251 [00:00<01:25,  2.94it/s]

This image is: <class 'numpy.ndarray'> with dimesions: (720, 1280, 3)
slope: 0.596638655462
x1: 957 y1: 606
x2: 1076 y2: 677
slope: 0.592592592593
x1: 984 y1: 610
x2: 1092 y2: 674
slope: -0.647887323944
x1: 249 y1: 686
x2: 604 y2: 456
slope: 0.601895734597
x1: 667 y1: 422
x2: 1089 y2: 676
slope: -0.525
x1: 560 y1: 482
x2: 640 y2: 440
slope: -0.623456790123
x1: 462 y1: 546
x2: 624 y2: 445
slope: 0.6
x1: 959 y1: 608
x2: 1074 y2: 677
slope: 0.8
x1: 714 y1: 444
x2: 779 y2: 496
slope: 0.554585152838
x1: 830 y1: 526
x2: 1059 y2: 653
slope: 0.926470588235
x1: 692 y1: 420
x2: 760 y2: 483
slope: 0.944444444444
x1: 971 y1: 678
x2: 1007 y2: 712
slope: 0.96699669967
x1: 695 y1: 418
x2: 998 y2: 711
slope: -0.6
x1: 463 y1: 544
x2: 638 y2: 439
slope: 0.601941747573
x1: 988 y1: 611
x2: 1091 y2: 673


...

This image is: <class 'numpy.ndarray'> with dimesions: (720, 1280, 3)
slope: 0.56
x1: 730 y1: 463
x2: 805 y2: 505
slope: 0.553627760252
x1: 468 y1: 323
x2: 1102 y2: 674
slope: -0.516129032258
x1: 592 y1: 466
x2: 654 y2: 434
slope: 0.6
x1: 495 y1: 326
x2: 1080 y2: 677
slope: 0.600306278714
x1: 497 y1: 325
x2: 1150 y2: 717
slope: 0.601851851852
x1: 1045 y1: 651
x2: 1153 y2: 716
slope: 0.555555555556
x1: 728 y1: 458
x2: 737 y2: 463
slope: 0.55223880597
x1: 452 y1: 311
x2: 519 y2: 348
slope: -0.6735218509
x1: 243 y1: 705
x2: 632 y2: 443
slope: 0.6
x1: 523 y1: 337
x2: 1153 y2: 715
slope: -0.69820971867
x1: 276 y1: 687
x2: 667 y2: 414
slope: 0.577777777778
x1: 861 y1: 547
x2: 1086 y2: 677


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████▌| 250/251 [01:27<00:00,  2.31it/s]

This image is: <class 'numpy.ndarray'> with dimesions: (720, 1280, 3)
slope: 0.6
x1: 522 y1: 335
x2: 1092 y2: 677
slope: 0.577352472089
x1: 465 y1: 314
x2: 1092 y2: 676
slope: -0.666666666667
x1: 554 y1: 492
x2: 623 y2: 446
slope: 0.6
x1: 521 y1: 333
x2: 906 y2: 564
slope: 0.602941176471
x1: 805 y1: 512
x2: 1077 y2: 676
slope: 0.599297012302
x1: 521 y1: 336
x2: 1090 y2: 677
slope: 0.509652509653
x1: 465 y1: 321
x2: 724 y2: 453
slope: -0.579545454545
x1: 562 y1: 485
x2: 650 y2: 434
slope: 0.583333333333
x1: 492 y1: 323
x2: 528 y2: 344
slope: 0.552272727273
x1: 464 y1: 322
x2: 904 y2: 565
slope: 0.623188405797
x1: 1042 y1: 676
x2: 1111 y2: 719
slope: 0.576530612245
x1: 892 y1: 564
x2: 1088 y2: 677
slope: 0.57805907173
x1: 502 y1: 326
x2: 739 y2: 463
slope: 0.509652509653
x1: 456 y1: 310
x2: 715 y2: 442
slope: 0.57753164557
x1: 457 y1: 311
x2: 1089 y2: 676


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [01:28<00:00,  2.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge1.mp4 

Wall time: 1min 29s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))